In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/27 04:10:33 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/27 04:10:33 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/27 04:10:33 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/27 04:10:33 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

In [3]:
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

In [4]:
training.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [5]:
lr = LogisticRegression(maxIter=30, regParam=0.01)

In [6]:
model = lr.fit(training)

23/04/27 04:10:46 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/27 04:10:46 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [7]:
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [8]:
prediction = model.transform(test)

In [9]:
prediction.show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0|[-1.0,1.5,1.3]|[-6.2435552280691...|[0.00193916797132...|       1.0|
|  0.0|[3.0,2.0,-0.1]|[5.45228378434145...|[0.99573179163953...|       0.0|
|  1.0|[0.0,2.2,-1.5]|[-4.4104198106252...|[0.01200422428416...|       1.0|
+-----+--------------+--------------------+--------------------+----------+

